In [2]:
import cv2 as cv
import numpy as np
import random

class ball_game:
    
    def __init__(self):
        #Environment settings
        self.width = 1000
        self.height = 600
        self.limits = [0,0,self.width,self.height]
        #Past bounding boxes of the balls
        self.past_bboxes = []
        #Bounding boxes of the balls
        self.bboxes = []
        #Trackers of the balls
        self.trackers = []
        #Id count numbers
        self.ids_count = 0
        #Id numbers
        self.ids = []
        #Boxes to be tracked
        self.balls = []
        #Adding balls
        self.balls.append(self.create_ball([int(self.width/2),int(self.height/2)],[10,3],[True,True],50,(0,0,255)))
        self.balls.append(self.create_ball([int(self.width/3),int(self.height/3)],[10,6],[True,True],50,(0,255,0)))
        #Run Code
        self.run()
        
    def create_ball(self,positions,speeds,directions,size,color):
        ball = [positions,speeds,directions,size,color]
        self.create_tracker(ball)
        return ball
    
    def create_tracker(self,ball):
        tracker = [[self.ids_count],[ball[0][0],ball[0][1],ball[0][0]+2*ball[3],ball[0][1]+2*ball[3]]]
        self.trackers.append(tracker)
        self.ids.append(self.ids_count)
        self.ids_count = self.ids_count+1
    
    def balls_manager(self):
        for i,ball in enumerate(self.balls):
            x = ball[0][0]
            y = ball[0][1]
            size = ball[-2]        
            if x+size>=self.limits[2] or x-size<=self.limits[0]:
                ball[1][0] = ball[1][0]*-1
            if y+size>=self.limits[3] or y-size<=self.limits[1]:
                ball[1][1] = ball[1][1]*-1                   
        for i,ball in enumerate(self.balls):
            ball[0][0] = ball[0][0] + ball[1][0]
            ball[0][1] = ball[0][1] + ball[1][1]
                
    def draw_balls(self,frame):
        for ball in self.balls:
            cv.circle(frame,(ball[0][0],ball[0][1]),ball[-2],ball[-1],-1)
            cv.rectangle(frame,(int(ball[0][0]-ball[-2]/1),int(ball[0][1]-ball[-2]/1)),(int(ball[0][0]+ball[-2]/1),int(ball[0][1]+ball[-2]/1)),(255,0,0),3)
        for tracker in self.trackers:
            cv.putText(frame,str(tracker[0]),(int(tracker[1][0]+20),int(tracker[1][1]+50)),cv.FONT_HERSHEY_SIMPLEX,1,(0,0,0),2)
        for past_boxes in self.past_bboxes:
            cv.rectangle(frame,(int(past_boxes[0]),int(past_boxes[1])),(int(past_boxes[2]),int(past_boxes[3])),(100,100,100),2)
    
    def update_bboxes(self):
        self.past_bboxes = self.bboxes.copy()
        self.bboxes.clear()
        for ball in self.balls:
            self.bboxes.append([int(ball[0][0]-ball[-2]/1),int(ball[0][1]-ball[-2]/1),int(ball[0][0]+ball[-2]/1),int(ball[0][1]+ball[-2]/1)])
    
    def update_trackers(self):
        new_trackers = []
        overlaps_bboxes = []
        for k,present_box in enumerate(self.bboxes):
            overlaps_bbox = []
            for i,past_box in self.trackers:
                x_dif = 0
                y_dif = 0
                past_area = abs(past_box[0]-past_box[2])*abs(past_box[1]-past_box[3])
                if present_box[0]>=past_box[0] and present_box[0]<=past_box[2]:
                    x_dif = abs(present_box[0]-past_box[2])
                elif present_box[2]>=past_box[0] and present_box[2]<=past_box[2]:
                    x_dif = abs(present_box[2]-past_box[0])
                if present_box[1]>=past_box[1] and present_box[1]<=past_box[3]:
                    y_dif = abs(present_box[1]-past_box[3])
                elif present_box[3]>=past_box[1] and present_box[3]<=past_box[3]:
                    y_dif = abs(present_box[3]-past_box[1])
                overlap = x_dif*y_dif/past_area
                overlaps_bbox.append([overlap,i,k])
            overlaps_bboxes.append(overlaps_bbox)
        ids_selected = []
        for over_bbox in overlaps_bboxes:
            if max(over_bbox)[1][0] not in ids_selected:
                tracker = [max(over_bbox)[1],self.bboxes[max(over_bbox)[-1]]]
                new_trackers.append(tracker)
                ids_selected.append(max(over_bbox)[1][0])
                continue
            if max(over_bbox)[1][0] in ids_selected:
                id_num = list(set((self.ids))-set((ids_selected)))[0]
                tracker = [[id_num],self.bboxes[max(over_bbox)[-1]]]
                new_trackers.append(tracker)
                ids_selected.append(id_num)
                continue
        #print(overlaps_bboxes)
        self.trackers = new_trackers.copy()
    
    def run(self):
        while(True):
            background = np.zeros((self.height,self.width,3))
            self.balls_manager()
            self.draw_balls(background)
            self.update_bboxes()
            self.update_trackers()
            cv.imshow('frame',background)
            # Press Q on keyboard to  exit 
            if cv.waitKey(1) & 0xFF == ord('q'): 
                break
            
        cv.destroyAllWindows()
ball_game()